In [1]:
import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time

In [2]:
# Get authentication credentials to initialize ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AWgavdcdJaT2CbSmGCt23CEA3rToYwqxT2SEs15aE8LeQljeJz4WRzlqMsY

Successfully saved authorization token.


In [29]:
# Define geometry for data collection


# Nanda Bet larger S1 extent
# roi = ee.Geometry.Rectangle([69.98568, 22.66953, 72.76651, 24.60271]);
# shp_file = 'data/s1_boundary.shp'
# boundary = geemap.shp_to_ee(shp_file)


# Nanda Bet smaller region
roi = ee.Geometry.Rectangle([70.7980547394194417, 23.2880385161501735, 71.5060515087354958, 23.9024834537972986])  
shp_file = 'data/smaller_region.shp'
boundary = geemap.shp_to_ee(shp_file)


In [30]:
# Get dem in roi

dem_col = ee.ImageCollection("projects/sat-io/open-datasets/GLO-30").filterBounds(roi)

In [31]:
# Display metadata

print(dem_col.first().getInfo())

{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [3600, 3600], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, 69.99986111111112, 0, -0.0002777777777777778, 24.000138888888888]}], 'version': 1644365862895597, 'id': 'projects/sat-io/open-datasets/GLO-30/Copernicus_DSM_COG_10_N23_00_E070_00_DEM', 'properties': {'num_bands': 1, 'id_no': 'Copernicus_DSM_COG_10_N23_00_E070_00_DEM', 'ysize': 3600, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[69.99970494982537, 24.000069041867576], [69.99969540881884, 24.00002321067513], [69.99969768048709, 23.000277787759522], [69.99982062316059, 23.00014947886318], [69.99993166227458, 22.999999441833708], [70.24993057565723, 22.999999525523897], [70.49986112194406, 22.999999534546852], [70.74979163226232, 22.999999547443565], [70.99972194905587, 22.99999949382943], [70.99986143326281, 23.000112560111326], [71.00002449731586, 23.000214662483387], [71.00002685530504

In [32]:
# Print number of images in image collection

print('Number of images in dem collection:', dem_col.size().getInfo())

Number of images in dem collection: 362


In [39]:
# Merge collection to image by taking mean and clip to boundary

dem = ee.Image(dem_col.mean()).clip(boundary.geometry())

In [35]:
# Define visualization parameters

palette = ['#000000', '#1E5BC0', '#0882E9', '#00A6D7', '#00C67B', '#31D36D', '#69E079',
           '#A5EE87', '#E5FC93', '#F3EF92', '#D1C680', '#B29E70', '#967860', '#88655D',
           '#A68A85', '#C2B1AF', '#E1D8D7', '#FFFFFF'];


dem_vis = {'min': 0, 'max': 50, 'palette': palette};

In [40]:
# Create map and add layers

M = geemap.Map()
M.centerObject(roi, 10)
M.addLayer(dem,dem_vis,'Elevation')
M

Map(center=[23.595422252032954, 71.15205312407748], controls=(WidgetControl(options=['position', 'transparent_…

In [46]:
# Export dem geotiff to google drive

txt = 'copernicus_dem30m_nanda_bet'
task = ee.batch.Export.image.toDrive(image=dem,
                                    description=txt,
                                    scale=30,
                                    region=boundary.geometry(),
                                    fileNamePrefix=txt,
                                    crs='EPSG:4326',
                                    fileFormat='GeoTIFF',
                                    maxPixels=1e9)
task.start()    

In [48]:
# Check task progress

task.list()

[<Task DTHEXSF4RMUDMEGZ5U7G5WFM EXPORT_IMAGE: copernicus_dem30m_nanda_bet (COMPLETED)>,
 <Task 4BT4OK562GZS4EUMXNG2JSCE INGEST_TABLE: Ingest table: "projects/bdowns-research/assets/nanda_bet_small_region" (COMPLETED)>,
 <Task TKNLIZICDY2PJO47PRZH6RLJ EXPORT_IMAGE: s1_nanda_bet_poly1_12-28-2020 (COMPLETED)>,
 <Task GR3LBYSXMBLZLO54G3Z3G3S5 EXPORT_IMAGE: s1_nanda_bet_poly1_12-16-2020 (COMPLETED)>,
 <Task DHOYXWD4HWPKWTCLWIQ3S2SI EXPORT_IMAGE: s1_nanda_bet_poly1_12-04-2020 (COMPLETED)>,
 <Task HORAZYWZQX5CJW52B7YGXIHA EXPORT_IMAGE: s1_nanda_bet_poly1_11-22-2020 (COMPLETED)>,
 <Task 5J7TJH7LUIIXYZGG7DGOGMQA EXPORT_IMAGE: s1_nanda_bet_poly1_11-10-2020 (COMPLETED)>,
 <Task 3XCE7RZLPKIWNDTIKAMM224W EXPORT_IMAGE: s1_nanda_bet_poly1_10-29-2020 (COMPLETED)>,
 <Task ICYCP57PQW4MPFSSB2VSGNMH EXPORT_IMAGE: s1_nanda_bet_poly1_10-17-2020 (COMPLETED)>,
 <Task AR6LEZ5N6O5J7RAPRF3OLEQR EXPORT_IMAGE: s1_nanda_bet_poly1_10-05-2020 (COMPLETED)>,
 <Task VAV4Y2CDHKGOWGI5E7OCGNTS EXPORT_IMAGE: s1_nanda_bet_po